<a href="https://colab.research.google.com/github/AokiMasataka/LSTM_sample/blob/fix_train/LSTM_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
pip install -U torch torchvision

Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.6.0+cu101)
Requirement already up-to-date: torchvision in /usr/local/lib/python3.6/dist-packages (0.7.0+cu101)


In [37]:
import torch
from torch import nn, functional as F
import numpy as np

VOCAB_SIZE = 64
EMBEDDING_DIM = 16
HIDDEN_DIM = 64
sequence = 5
BATCH_SIZE = 1

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.word_embeddings = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
        self.lstm = nn.LSTM(input_size=EMBEDDING_DIM, hidden_size=HIDDEN_DIM, batch_first=True, num_layers=1)
        self.liner = nn.Linear(HIDDEN_DIM, VOCAB_SIZE)
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        return (torch.zeros(1, BATCH_SIZE, HIDDEN_DIM),
                torch.zeros(1, BATCH_SIZE, HIDDEN_DIM))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        embeds = embeds.view(BATCH_SIZE, -1, EMBEDDING_DIM)
        output, self.hidden = self.lstm(embeds, self.hidden)
        output = output.view(-1, HIDDEN_DIM)
        output = self.liner(output)
        return output

In [32]:
def getData(batch_size=BATCH_SIZE):
  tensor = torch.zeros(sequence).reshape(1, -1)
  for i in range(batch_size):
    numbers = []
    num = np.random.randint(0, VOCAB_SIZE - sequence - 1)
    for i in range(sequence):
      numbers.append(num + i)
    tensor = torch.cat((tensor, torch.tensor(numbers).reshape(1, -1)), 0)
  return tensor[1:].to(torch.long)

In [48]:
decoder = Decoder()
CEL = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(decoder.parameters(), lr=0.01, momentum=0.9)

# 学習
decoder.train()
for i in range(2000):
  data = getData()
  x = data
  y = data.reshape((-1)) + 1
  decoder.hidden = decoder.init_hidden()
  output = decoder(x)
  loss = CEL(output, y)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  if(i + 1) % 100 == 0:
    print("epoch :", loss.item())


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch : 3.8938605785369873
epoch : 3.458599090576172
epoch : 3.6787288188934326
epoch : 3.4791998863220215
epoch : 2.4073848724365234
epoch : 1.8666175603866577
epoch : 1.6187750101089478
epoch : 0.8628924489021301
epoch : 0.584494948387146
epoch : 0.4138711988925934
epoch : 0.2409854382276535
epoch : 0.17669174075126648
epoch : 0.2226695567369461
epoch : 0.11096011102199554
epoch : 0.1533803641796112
epoch : 0.14232398569583893
epoch : 0.1313129961490631
epoch : 0.08420880883932114
epoch : 0.08191785216331482
epoch : 0.06722093373537064


In [64]:
# Test
decoder.eval()
start = [[5]]
test_tensor = torch.tensor(start)
decoder.hidden = decoder.init_hidden()
for i in range(sequence):
  output = decoder(test_tensor[:, -1].reshape(1, 1))
  number = torch.argmax(output).reshape(1, 1)
  test_tensor = torch.cat((test_tensor, number), -1)
  print(test_tensor[0])

tensor([5, 6])
tensor([5, 6, 7])
tensor([5, 6, 7, 8])
tensor([5, 6, 7, 8, 9])
tensor([ 5,  6,  7,  8,  9, 10])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
